### 2. 练习
#### 【练习一】有2张公司的员工信息表，每个公司共有16名员工，共有五个公司，请解决如下问题：

#### (a) 每个公司有多少员工满足如下条件：既出现第一张表，又出现在第二张表。
#### (b) 将所有不符合(a)中条件的行筛选出来，合并为一张新表，列名与原表一致。
#### (c) 现在需要编制所有80位员工的信息表，对于(b)中的员工要求不变，对于满足(a)条件员工，它们在某个指标的数值，取偏离它所属公司中满足(b)员工的均值数较小的哪一个，例如：P公司在两张表的交集为{p1}，并集扣除交集为{p2,p3,p4}，那么如果后者集合的工资均值为1万元，且p1在表1的工资为13000元，在表2的工资为9000元，那么应该最后取9000元作为p1的工资，最后对于没有信息的员工，利用缺失值填充。

In [2]:
import pandas as pd

In [3]:
pd.read_csv('data/Employee1.csv').head()

,Company,Name,Age,Height,Weight,Salary
0,A,a1,47,188,63.7,25819
1,A,a3,39,172,55.9,21983
2,A,a4,43,158,62.5,21755
3,A,a6,42,182,76.9,17354
4,A,a7,49,171,94.6,6177


In [5]:
data1 = pd.read_csv('data/Employee1.csv')
data2 = pd.read_csv('data/Employee2.csv')
data1

,Company,Name,Age,Height,Weight,Salary
0,A,a1,47,188,63.7,25819
1,A,a3,39,172,55.9,21983
2,A,a4,43,158,62.5,21755
3,A,a6,42,182,76.9,17354
4,A,a7,49,171,94.6,6177
5,A,a8,51,168,89.5,3246
6,A,a9,36,186,62.8,3569
7,A,a13,58,190,75.9,21854
8,A,a15,35,162,97.2,4706
9,A,a16,35,157,77.6,16130


In [9]:
name = list(set(data1['Name']).intersection(set(data2['Name'])))
name

['d10',
 'c13',
 'c10',
 'c3',
 'e10',
 'a3',
 'b15',
 'b3',
 'e11',
 'b1',
 'b7',
 'e8',
 'c12',
 'a6',
 'd5',
 'a1']

In [10]:
t1 = data1[~data1['Name'].isin(name)]
t2 = data2[~data2['Name'].isin(name)]
t3 = pd.concat([t2,t2]).set_index('Name')
t3.head(5)

,Company,Age,Height,Weight,Salary
Name,,,,,
a2,A,50,190,83.4,6673
a5,A,51,176,97.2,23294
a10,A,56,164,82.2,24799
a12,A,36,177,96.9,10270
a14,A,56,190,62.6,10229


In [12]:
data1['重复'] = ['Y_1' if data1.loc[i,'Name'] in name else 'N' for i in range(data1.shape[0])]
data2['重复'] = ['Y_2' if data2.loc[i,'Name'] in name else 'N' for i in range(data2.shape[0])]
data1 = data1.set_index(['Name','重复'])
data2 = data2.set_index(['Name','重复'])
df_c = pd.concat([data1,data2])
result = pd.DataFrame({'Company':[],'Name':[],'Age':[],'Height':[],'Weight':[],'Salary':[]})
group = df_c.groupby(['Company','重复'])
for i in name:
    first = group.get_group((i[0].upper(),'Y_1')).reset_index(level=1).loc[i,:][-4:]
    second = group.get_group((i[0].upper(),'Y_2')).reset_index(level=1).loc[i,:][-4:]
    mean = group.get_group((i[0].upper(),'N')).reset_index(level=1).mean()
    final = [i[0].upper(),i]
    for j in range(4):
        final.append(first[j] if abs(first[j]-mean[j])<abs(second[j]-mean[j]) else second[j])
    result = pd.concat([result,pd.DataFrame({result.columns.tolist()[k]:[final[k]] for k in range(6)})])
result = pd.concat([result.set_index('Name'),t3])
for i in list('abcde'):
    for j in range(1,17):
        item = i+str(j)
        if item not in result.index:
            result = pd.concat([result,pd.DataFrame({'Company':[i.upper()],'Name':[item]
                 ,'Age':[np.nan],'Height':[np.nan],'Weight':[np.nan],'Salary':[np.nan]}).set_index('Name')])
result['Number'] = [int(i[1:]) for i in result.index]
result.reset_index().drop(columns='Name').set_index(['Company','Number']).sort_index()

KeyError: 0

#### 【练习二】有2张课程的分数表（分数随机生成），但专业课（学科基础课、专业必修课、专业选修课）与其他课程混在一起，请解决如下问题：

In [13]:
pd.read_csv('data/Course1.csv').head()

,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0


#### (a) 将两张表分别拆分为专业课与非专业课（结果为四张表）。
#### (b) 将两张专业课的分数表和两张非专业课的分数表分别合并。
#### (c) 不使用(a)中的步骤，请直接读取两张表合并后拆分。
#### (d) 专业课程中有缺失值吗，如果有的话请在完成(3)的同时，用组内（3种类型的专业课）均值填充缺失值后拆分。

In [14]:
df1 = pd.read_csv('data/Course1.csv')
df2 = pd.read_csv('data/Course2.csv')
df_a11,df_a12,df_a21,df_a22 =0,0,0,0
df_a11= df1.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')
df_a12= df1.query('课程类别 not in ["学科基础课","专业必修课","专业选修课"]')
df_a21= df2.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')
df_a22= df2.query('课程类别 not in ["学科基础课","专业必修课","专业选修课"]')
df_a11.head()

,课程名字,课程类别,学分,分数
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0
6,数据结构与算法,学科基础课,5,82.0


In [15]:
special = pd.concat([df_a11,df_a21])
common = pd.concat([df_a12,df_a22])
special.query('课程类别 not in ["学科基础课","专业必修课","专业选修课"]')

,课程名字,课程类别,学分,分数


In [16]:
common.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')

,课程名字,课程类别,学分,分数


In [17]:
common.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')

,课程名字,课程类别,学分,分数


In [19]:
df = pd.concat([df1,df2])
special2 = df.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')
common2 = df.query('课程类别 not in ["学科基础课","专业必修课","专业选修课"]')
(special.equals(special2),common.equals(common2))

(True, True)

In [20]:
df['分数'] = df.groupby('课程类别').transform(lambda x: x.fillna(x.mean()))['分数']
df.isnull().all()

课程名字    False
课程类别    False
学分      False
分数      False
dtype: bool

In [21]:
special3 = df.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')
common3 = df.query('课程类别 not in ["学科基础课","专业必修课","专业选修课"]')
common3.head()

,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0
